## 기본 셋팅

In [83]:
import math
import collections

import numpy as np
import sympy as sy
import matplotlib.pyplot as plt

from scipy.optimize import fsolve

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [84]:
# Connect to the simulator
client = RemoteAPIClient()
sim = client.require("sim")

print("Connected to CoppeliaSim")

Connected to CoppeliaSim


In [85]:
# Get arm's joints
joints = []
for i in range(5):
    joints.append(sim.getObject(f"/youBotArmJoint{i}"))

# Gripper joint
joints.append(sim.getObject(f"/youBotGripperJoint2"))

points = []

# Car reference frame
points.append(sim.getObject(f"/youBot_ref"))
# Joint-0 위치
points.append(sim.getObject(f"/p0_ref"))
# End Effector 위치
points.append(sim.getObject(f"/pe_ref"))
# Target 위치
points.append(sim.getObject(f"/Target"))

# joint 제어 모드 변경
for joint in joints:
    sim.setObjectInt32Param(
        joint,
        sim.jointintparam_dynctrlmode,
        sim.jointdynctrl_position,
    )

In [86]:
# joint angle 조회
def read_joints(joints):
    js = []
    for joint in joints:
        j = sim.getJointPosition(joint)
        js.append(j)
    return js

print("Initial joint angles:", read_joints(joints))

Initial joint angles: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [87]:
# point position & orientation 조회
def read_points(points):
    ps = []
    for point in points:
        p = sim.getObjectPosition(point)
        o = sim.getObjectOrientation(point)
        ps.append(np.array(p + o))
    return ps

def print_points(ps):
    for i, p in enumerate(ps):
        print(f"Point {i+1}: Position = {p[:3]}, Orientation = {p[3:]}")

ps = read_points(points)
print("Initial point positions and orientations:")
print_points(ps)

Initial point positions and orientations:
Point 1: Position = [2.50000000e-02 9.99960000e-01 1.53971835e-11], Orientation = [ 3.71924713e-15 -1.44328993e-15 -1.66533454e-16]
Point 2: Position = [0.0249  1.16618 0.099  ], Orientation = [-5.55111512e-17 -1.11022302e-16  5.55111512e-17]
Point 3: Position = [0.0249  1.19918 0.74   ], Orientation = [-2.52598440e-16 -4.22626393e-16  4.79537226e-17]
Point 4: Position = [0.075 1.6   0.02 ], Orientation = [-0.  0. -0.]


In [88]:
# joint 제어
def control_joint(joints, thetas):
    for joint, j in zip(joints, thetas):
        sim.setJointTargetPosition(joint, j)

In [89]:
# gripper 제어
def control_gripper(gripper, state):
    position = sim.getJointPosition(gripper)
    position += 0.005 if state else -0.005
    sim.setJointTargetPosition(gripper, position)
    return position

In [90]:
# control joint velocity

def trace_arm(joints, target_thetas):
    js = read_joints(joints)
    diff_sum = 0
    thetas = []
    for i, target in enumerate(target_thetas):
        diff = js[i] - target
        diff_sum += diff
        thetas.append(js[i] - min(0.1, max(-0.05, diff)))
    control_joint(joints, thetas)
    return diff_sum

In [91]:
# forward kinematics
def fk(thetas, params):
    j1, j2, j3, j4 = thetas[:4]
    j0, pc = params[:2]
    # 월드 기준 자동차
    dc = pc[5]
    TWC = np.array([
        [np.cos(dc), -np.sin(dc), 0, pc[0]],
        [np.sin(dc),  np.cos(dc), 0, pc[1]],
        [         0,           0, 1, pc[2]],
        [         0,           0, 0,     1]
    ])

    # 자동차 -> joint-0
    TC0 = np.array([ # 좌표이동 및 y축을 기준으로 90도 회전
        [1, 0, 0, 0.0],
        [0, 1, 0, 0.166],
        [0, 0, 1, 0.099],
        [0, 0, 0, 1]
    ]) @ np.array([
        [np.cos(j0), -np.sin(j0), 0, 0],
        [np.sin(j0),  np.cos(j0), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW0 = TWC @ TC0

    # joint-0 -> joint-1
    ay1 = np.pi / 2
    T01 = np.array([ # 좌표이동 및 y축을 기준으로 90도 회전
        [ np.cos(ay1), 0, np.sin(ay1), 0.0],
        [           0, 1,           0, 0.033],
        [-np.sin(ay1), 0, np.cos(ay1), 0.147],
        [           0, 0,           0, 1]
    ]) @ np.array([ # z축을 기준으로 j1만큼 회전
        [np.cos(j1), -np.sin(j1), 0, 0],
        [np.sin(j1),  np.cos(j1), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW1 = TW0 @ T01

    # joint-1 -> joint-2
    T12 = np.array([ # 좌표이동, 회전 없음
        [1, 0, 0, -0.155],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([ # z축을 기준으로 j2만큼 회전
        [np.cos(j2), -np.sin(j2), 0, 0],
        [np.sin(j2),  np.cos(j2), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW2 = TW1 @ T12

    # joint-2 -> joint-3
    T23 = np.array([ # 좌표이동, 회전 없음
        [1, 0, 0, -0.135],
        [0, 1, 0,  0.0],
        [0, 0, 1,  0.0],
        [0, 0, 0,  1]
    ]) @ np.array([ # z축을 기준으로 j3만큼 회전
        [np.cos(j3), -np.sin(j3), 0, 0],
        [np.sin(j3),  np.cos(j3), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW3 = TW2 @ T23

    # joint-3 -> joint-4
    ay4 = -np.pi / 2
    T34 = np.array([ # 좌표이동 및 y축을 기준으로 -90도 회전
        [ np.cos(ay4), 0, np.sin(ay4), -0.081],
        [           0, 1,           0,  0.0],
        [-np.sin(ay4), 0, np.cos(ay4),  0.0],
        [           0,  0,          0,  1]
    ]) @ np.array([ # z축을 기준으로 j4만큼 회전
        [np.cos(j4), -np.sin(j4), 0, 0],
        [np.sin(j4),  np.cos(j4), 0, 0],
        [         0,           0, 1, 0],
        [         0,           0, 0, 1]
    ])
    TW4 = TW3 @ T34

    pe_hat = TW4 @ np.array([ 0.0,   0.0,   0.123, 1])

    return pe_hat[:3]

In [92]:
# inverse kinematics
def ik(thetas, params):  
    pt = params[-1][:3]
    pe_hat = fk(thetas, params)
    # theta 범위 검증
    if thetas[0] < np.deg2rad(-90) or np.deg2rad(75) < thetas[0]:
        return 10, 0, 0, 0
    elif thetas[1] < np.deg2rad(-131.00) or np.deg2rad(131.00) < thetas[1]:
        return 10, 0, 0, 0
    elif thetas[2] < np.deg2rad(-102.00) or np.deg2rad(102.00) < thetas[2]:
        return 10, 0, 0, 0
    elif thetas[3] < np.deg2rad(-90.00) or np.deg2rad(90.00) < thetas[3]:
        return 10, 0, 0, 0
    return np.linalg.norm(pe_hat - pt), 0, 0, 0

In [93]:
def solve(js, ps):
    p0, pt = ps[1], ps[-1]
    diff = pt[:2] - p0[:2]
    angle = math.atan2(diff[1], diff[0])
    j0 = angle - p0[-1] - np.pi / 2
    target_thetas = fsolve(
        ik,
        [js[1], js[2], js[3], js[4]],
        [j0, ps[0], ps[-1]]
    )
    return np.concatenate((np.array([j0]), target_thetas))

In [94]:
n_steps = 400

# 시뮬레이션 실행
sim.setStepping(True)
sim.startSimulation()

# 각 스텝 실행
js = read_joints(joints)
ps = read_points(points)
target_thetas = solve(js, ps)
place_thetas = [np.pi, -np.pi / 6, -np.pi / 2.7, -np.pi / 3, 0]
base_thetas = [0, 0, 0, 0, 0]
stage = 0
gripper_positions = collections.deque(maxlen=100)
for i in range(n_steps):
    if stage == 0: # move to target
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 1
    elif stage == 1: # grip target
        position = control_gripper(joints[-1], True)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 2
    elif stage == 2:  # pick the target
        target_thetas[0] = np.pi
        target_thetas[1] = -np.pi / 12
        target_thetas[2] = -np.pi / 6
        diff_sum = trace_arm(joints, target_thetas)
        if abs(diff_sum) < 0.005:
            stage = 3
    elif stage == 3: # move to place
        diff_sum = trace_arm(joints, place_thetas)
        if abs(diff_sum) < 0.005:
            stage = 4
    elif stage == 4: # place the target
        position = control_gripper(joints[-1], False)
        gripper_positions.append(position)
        if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
            stage = 5
    elif stage == 5: # move to base
        diff_sum = trace_arm(joints, base_thetas)
        if abs(diff_sum) < 0.005:
            stage = 6
    sim.step()

# 시뮬레이션 종료
sim.stopSimulation()

C:\Users\joy4m\AppData\Local\Temp\ipykernel_40664\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


# Test 2

## Sampling

In [70]:
def sample_configuration_space(n_samples):
    """Generate random samples within joint limits"""
    joint_ranges = [
        (-np.pi, np.pi),          # Joint 0: -180 to 180 degrees
        (-np.pi/2, np.pi*75/180), # Joint 1: -90 to 75 degrees
        (-np.pi*131/180, np.pi*131/180), # Joint 2: -131 to 131 degrees
        (-np.pi*102/180, np.pi*102/180), # Joint 3: -102 to 102 degrees
        (-np.pi/2, np.pi/2)       # Joint 4: -90 to 90 degrees
    ]
    
    samples = []
    for _ in range(n_samples):
        config = []
        for min_val, max_val in joint_ranges:
            config.append(np.random.uniform(min_val, max_val))
        samples.append(config)
    
    return np.array(samples)

In [71]:
def create_dummy(sim, position, name):
    """Create a dummy object in CoppeliaSim at specified position"""
    # Convert numpy array to list for CoppeliaSim API
    if isinstance(position, np.ndarray):
        position = position.tolist()
        
    dummy_handle = sim.createDummy(0.05)
    sim.setObjectPosition(dummy_handle, -1, position)
    sim.setObjectAlias(dummy_handle, f"Sample_{name}")
    return dummy_handle

In [72]:
def visualize_samples(sim, samples):
    """Visualize samples by creating dummies at end-effector positions"""
    dummy_handles = []
    
    # Get car position and orientation for base transform
    car_handle = sim.getObject("/youBot_ref")
    car_position = sim.getObjectPosition(car_handle, -1)
    car_orientation = sim.getObjectOrientation(car_handle, -1)
    
    # Convert samples to list if it's numpy array
    if isinstance(samples, np.ndarray):
        samples_list = samples.tolist()
    else:
        samples_list = samples
    
    for i, sample in enumerate(samples_list):
        # 각 샘플의 joint 0 값을 사용
        base_params = [sample[0], [*car_position, *car_orientation], None]
        
        # Calculate forward kinematics for this configuration
        pe_hat = fk(sample, base_params)
        
        # Create dummy at the calculated position
        dummy_handle = create_dummy(sim, pe_hat, f"{i}")
        dummy_handles.append(dummy_handle)
    
    return dummy_handles

In [ ]:
n_steps = 500
n_samples = 1000

# Start simulation
sim.setStepping(True)
sim.startSimulation()

# Generate and visualize samples
samples = sample_configuration_space(n_samples)
dummy_handles = visualize_samples(sim, samples)

# Step simulation to ensure visualization
for _ in range(10):
    sim.step()

# Stop simulation
#sim.stopSimulation()

' n_steps = 500\nn_samples = 1000\n\n# Start simulation\nsim.setStepping(True)\nsim.startSimulation()\n\n# Generate and visualize samples\nsamples = sample_configuration_space(n_samples)\ndummy_handles = visualize_samples(sim, samples)\n\n# Step simulation to ensure visualization\nfor _ in range(10):\n    sim.step()\n\n# Stop simulation\n#sim.stopSimulation() '

## Build Road Map

In [74]:
def check_collision(sim, config, obstacle_handle):
    """Check if a configuration collides with obstacles"""
    # Get car position and orientation for base transform
    car_handle = sim.getObject("/youBot_ref")
    car_position = sim.getObjectPosition(car_handle, -1)
    car_orientation = sim.getObjectOrientation(car_handle, -1)
    base_params = [config[0], [*car_position, *car_orientation], None]
    
    # Get handles for all robot parts
    robot_parts = [
        sim.getObject("/Rectangle22"),
        sim.getObject("/Rectangle10"),
        sim.getObject("/Rectangle9"),
        sim.getObject("/Rectangle8"),
        sim.getObject("/Rectangle7"),
        sim.getObject("/Rectangle6"),
        sim.getObject("/Rectangle5")
    ]
    
    # Update robot configuration
    for i, joint in enumerate(joints[:len(config)]):
        sim.setJointPosition(joint, config[i])
    
    # Check collision between each robot part and the obstacle
    for part in robot_parts:
        isColliding = check_distance_to_sphere(
            sim, 
            part,
            obstacle_handle,
            0.2  # sphere radius
        )
        if isColliding:
            return True
            
    return False

def check_distance_to_sphere(sim, part_handle, obstacle_handle, sphere_radius):
    """Check if a robot part collides with a spherical obstacle"""
    # Get positions
    part_pos = sim.getObjectPosition(part_handle, -1)
    obstacle_pos = sim.getObjectPosition(obstacle_handle, -1)
    
    # Get minimal distance between part and obstacle center
    dx = part_pos[0] - obstacle_pos[0]
    dy = part_pos[1] - obstacle_pos[1]
    dz = part_pos[2] - obstacle_pos[2]
    distance = np.sqrt(dx*dx + dy*dy + dz*dz)
    
    # Simple sphere collision check
    # Note: This is a simplified check. For more accurate results,
    # you might want to consider the actual shape of the robot parts
    return distance < sphere_radius

In [75]:
def find_k_nearest(samples, k):
    """
    Find k nearest neighbors for each configuration in samples
    
    Parameters:
    samples: numpy array of shape (n_samples, n_joints)
    k: number of neighbors to find
    
    Returns:
    nearest_indices: numpy array of shape (n_samples, k) containing indices of k nearest neighbors
    """
    n_samples = len(samples)
    k = min(k, n_samples - 1)  # k shouldn't be larger than n_samples - 1
    
    # 모든 sample 쌍 사이의 거리 계산
    distances = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(i+1, n_samples):
            # Calculate distance in configuration space
            # Joint angles의 경우 angular distance 사용
            dist = np.sqrt(
                np.sum((samples[i, :3] - samples[j, :3]) ** 2) +  # position distance
                np.sum(np.arctan2(np.sin(samples[i, 3:] - samples[j, 3:]), 
                                np.cos(samples[i, 3:] - samples[j, 3:])) ** 2)  # angular distance
            )
            distances[i, j] = dist
            distances[j, i] = dist
    
    # 각 sample에 대해 k개의 가장 가까운 이웃의 인덱스 찾기
    nearest_indices = np.zeros((n_samples, k), dtype=int)
    for i in range(n_samples):
        # 자기 자신을 제외한 가장 가까운 k개의 이웃 찾기
        nearest = np.argsort(distances[i])[1:k+1]  # 0번째는 자기 자신이므로 제외
        nearest_indices[i] = nearest
    
    return nearest_indices

In [76]:
from scipy.spatial import cKDTree

def find_k_nearest_kdtree(samples, k):
    """
    Find k nearest neighbors using KD-Tree (faster but simpler distance metric)
    """
    tree = cKDTree(samples)
    _, indices = tree.query(samples, k=k+1)  # k+1 because first match is self
    return indices[:, 1:]  # Remove self from neighbors

In [77]:
def visualize_edge(sim, start_pos, end_pos):
    """Draw a single edge between two configurations"""
    return sim.addDrawingObject(
        sim.drawing_lines, 2, 0, -1, 0, [0, 1, 0],  # Green line
        [start_pos[0], start_pos[1], start_pos[2],
         end_pos[0], end_pos[1], end_pos[2]]
    )

## Append Start and goal

In [78]:
def create_prm_with_start_goal(sim, start_config, goal_config, n_samples=100, k_neighbors=5):
    """Create PRM with start and goal configurations"""
    # Start simulation in stepping mode
    sim.setStepping(True)
    sim.startSimulation()
    
    # Generate samples
    samples = sample_configuration_space(n_samples)
    
    # Add start and goal configurations to samples
    samples = np.vstack([samples, start_config, goal_config])
    
    # Get obstacle handle
    obstacle_handle = sim.getObject("/Obstacle")
    
    # Build roadmap with visualization
    roadmap, valid_samples, valid_indices, dummy_handles, line_handles = \
        build_roadmap_with_start_goal(sim, samples, obstacle_handle, k_neighbors, n_samples)
    
    return roadmap, valid_samples, valid_indices, dummy_handles, line_handles

def build_roadmap_with_start_goal(sim, samples, obstacle_handle, k_neighbors=5, n_original_samples=100):
    """Build PRM roadmap including start and goal configurations"""
    roadmap = {}
    valid_samples = []
    valid_indices = []
    dummy_handles = []
    line_handles = []
    
    print("Checking samples for collision...")
    # Process samples including start and goal
    for i, sample in enumerate(samples):
        if not check_collision(sim, sample, obstacle_handle):
            valid_samples.append(sample)
            valid_indices.append(i)
            roadmap[i] = []
            
            # Get end effector position for visualization
            car_handle = sim.getObject("/youBot_ref")
            car_position = sim.getObjectPosition(car_handle, -1)
            car_orientation = sim.getObjectOrientation(car_handle, -1)
            base_params = [sample[0], [*car_position, *car_orientation], None]
            ee_pos = fk(sample.tolist(), base_params)
            
            # Visualize sample with different colors for start and goal
            if i == n_original_samples:  # Start configuration
                dummy_handle = create_colored_dummy(sim, ee_pos, "start", [1, 0, 0])  # Red
                print("Start configuration added")
            elif i == n_original_samples + 1:  # Goal configuration
                dummy_handle = create_colored_dummy(sim, ee_pos, "goal", [0, 0, 1])  # Blue
                print("Goal configuration added")
            else:
                dummy_handle = create_dummy(sim, ee_pos, f"sample_{i}")
            
            dummy_handles.append(dummy_handle)
            sim.step()
    
    valid_samples = np.array(valid_samples)
    print(f"Found {len(valid_samples)} collision-free samples")
    
    print("Finding nearest neighbors...")
    nearest_neighbors = find_k_nearest(valid_samples, k_neighbors)
    
    print("Building and visualizing roadmap...")
    # Connect nodes with collision-free paths
    for i, neighbors in enumerate(nearest_neighbors):
        current_config = valid_samples[i]
        car_position = sim.getObjectPosition(car_handle, -1)
        car_orientation = sim.getObjectOrientation(car_handle, -1)
        base_params = [current_config[0], [*car_position, *car_orientation], None]
        start_pos = fk(current_config.tolist(), base_params)
        
        for neighbor_idx in neighbors:
            neighbor_config = valid_samples[neighbor_idx]
            
            # Check intermediate configurations for collision
            collision = False
            delta = (neighbor_config - current_config) / 10  # n_collision_check = 10
            
            for step in range(1, 10):
                intermediate_config = current_config + delta * step
                if check_collision(sim, intermediate_config, obstacle_handle):
                    collision = True
                    break
            
            if not collision:
                roadmap[valid_indices[i]].append(valid_indices[neighbor_idx])
                
                # Visualize edge
                base_params = [neighbor_config[0], [*car_position, *car_orientation], None]
                end_pos = fk(neighbor_config.tolist(), base_params)
                
                # Different color for edges connected to start/goal
                if i == len(valid_samples)-2 or neighbor_idx == len(valid_samples)-2:  # Start
                    line_handle = visualize_colored_edge(sim, start_pos, end_pos, [1, 0, 0])
                elif i == len(valid_samples)-1 or neighbor_idx == len(valid_samples)-1:  # Goal
                    line_handle = visualize_colored_edge(sim, start_pos, end_pos, [0, 0, 1])
                else:
                    line_handle = visualize_edge(sim, start_pos, end_pos)
                
                line_handles.append(line_handle)
                sim.step()
    
    print("Roadmap construction completed")
    return roadmap, valid_samples, valid_indices, dummy_handles, line_handles

def create_colored_dummy(sim, position, name, color):
    """Create a colored dummy object in CoppeliaSim"""
    dummy_handle = sim.createDummy(0.1)  # Slightly larger for visibility
    sim.setObjectPosition(dummy_handle, -1, position)
    sim.setObjectAlias(dummy_handle, f"Sample_{name}")
    # Set dummy color (if supported by your CoppeliaSim version)
    try:
        sim.setShapeColor(dummy_handle, None, 0, color)
    except:
        pass
    return dummy_handle

def visualize_colored_edge(sim, start_pos, end_pos, color):
    """Draw a colored edge between two configurations"""
    return sim.addDrawingObject(
        sim.drawing_lines, 2, 0, -1, 0, color,
        [start_pos[0], start_pos[1], start_pos[2],
         end_pos[0], end_pos[1], end_pos[2]]
    )

In [79]:

def create_colored_dummy(sim, position, name, color):
    """Create a colored dummy object in CoppeliaSim"""
    # Convert numpy array to list if needed
    if isinstance(position, np.ndarray):
        position = position.tolist()
        
    dummy_handle = sim.createDummy(0.1)  # Slightly larger for visibility
    sim.setObjectPosition(dummy_handle, -1, position)
    sim.setObjectAlias(dummy_handle, f"Sample_{name}")
    # Set dummy color (if supported by your CoppeliaSim version)
    try:
        sim.setShapeColor(dummy_handle, None, 0, color)
    except:
        pass
    return dummy_handle

def visualize_colored_edge(sim, start_pos, end_pos, color):
    """Draw a colored edge between two configurations"""
    # Convert numpy arrays to lists if needed
    if isinstance(start_pos, np.ndarray):
        start_pos = start_pos.tolist()
    if isinstance(end_pos, np.ndarray):
        end_pos = end_pos.tolist()
        
    return sim.addDrawingObject(
        sim.drawing_lines, 2, 0, -1, 0, color,
        [start_pos[0], start_pos[1], start_pos[2],
         end_pos[0], end_pos[1], end_pos[2]]
    )

def build_roadmap_with_start_goal(sim, samples, obstacle_handle, k_neighbors=5, n_original_samples=100):
    """Build PRM roadmap including start and goal configurations"""
    roadmap = {}
    valid_samples = []
    valid_indices = []
    dummy_handles = []
    line_handles = []
    
    print("Checking samples for collision...")
    # Convert samples to list if it's numpy array
    if isinstance(samples, np.ndarray):
        samples = samples.tolist()
        
    # Process samples including start and goal
    for i, sample in enumerate(samples):
        if not check_collision(sim, sample, obstacle_handle):
            valid_samples.append(sample)
            valid_indices.append(i)
            roadmap[i] = []
            
            # Get end effector position for visualization
            car_handle = sim.getObject("/youBot_ref")
            car_position = sim.getObjectPosition(car_handle, -1)
            car_orientation = sim.getObjectOrientation(car_handle, -1)
            base_params = [sample[0], [*car_position, *car_orientation], None]
            ee_pos = fk(sample, base_params)
            
            # Visualize sample with different colors for start and goal
            if i == n_original_samples:  # Start configuration
                dummy_handle = create_colored_dummy(sim, ee_pos, "start", [1, 0, 0])  # Red
                print("Start configuration added")
            elif i == n_original_samples + 1:  # Goal configuration
                dummy_handle = create_colored_dummy(sim, ee_pos, "goal", [0, 0, 1])  # Blue
                print("Goal configuration added")
            else:
                dummy_handle = create_dummy(sim, ee_pos, f"sample_{i}")
            
            dummy_handles.append(dummy_handle)
            sim.step()
    
    valid_samples = np.array(valid_samples)
    print(f"Found {len(valid_samples)} collision-free samples")
    
    print("Finding nearest neighbors...")
    nearest_neighbors = find_k_nearest(valid_samples, k_neighbors)
    
    print("Building and visualizing roadmap...")
    # Connect nodes with collision-free paths
    for i, neighbors in enumerate(nearest_neighbors):
        current_config = valid_samples[i].tolist()  # Convert to list
        car_position = sim.getObjectPosition(car_handle, -1)
        car_orientation = sim.getObjectOrientation(car_handle, -1)
        base_params = [current_config[0], [*car_position, *car_orientation], None]
        start_pos = fk(current_config, base_params)
        
        for neighbor_idx in neighbors:
            neighbor_config = valid_samples[neighbor_idx].tolist()  # Convert to list
            
            # Check intermediate configurations for collision
            collision = False
            delta = (valid_samples[neighbor_idx] - valid_samples[i]) / 10
            
            for step in range(1, 10):
                intermediate_config = (valid_samples[i] + delta * step).tolist()
                if check_collision(sim, intermediate_config, obstacle_handle):
                    collision = True
                    break
            
            if not collision:
                roadmap[valid_indices[i]].append(valid_indices[neighbor_idx])
                
                # Visualize edge
                base_params = [neighbor_config[0], [*car_position, *car_orientation], None]
                end_pos = fk(neighbor_config, base_params)
                
                # Different color for edges connected to start/goal
                if i == len(valid_samples)-2 or neighbor_idx == len(valid_samples)-2:  # Start
                    line_handle = visualize_colored_edge(sim, start_pos, end_pos, [1, 0, 0])
                elif i == len(valid_samples)-1 or neighbor_idx == len(valid_samples)-1:  # Goal
                    line_handle = visualize_colored_edge(sim, start_pos, end_pos, [0, 0, 1])
                else:
                    line_handle = visualize_edge(sim, start_pos, end_pos)
                
                line_handles.append(line_handle)
                sim.step()
    
    print("Roadmap construction completed")
    return roadmap, valid_samples, valid_indices, dummy_handles, line_handles

## Path 

In [80]:
def find_path_in_roadmap(roadmap, valid_indices, start_idx, goal_idx):
    """Find path from start to goal in the roadmap using Dijkstra's algorithm"""
    from collections import defaultdict
    import heapq
    
    # Initialize distances and predecessors
    distances = defaultdict(lambda: float('infinity'))
    predecessors = {}
    distances[start_idx] = 0
    pq = [(0, start_idx)]
    visited = set()
    
    while pq:
        current_distance, current_vertex = heapq.heappop(pq)
        
        if current_vertex == goal_idx:
            break
            
        if current_vertex in visited:
            continue
            
        visited.add(current_vertex)
        
        # Check all neighbors
        for neighbor in roadmap[current_vertex]:
            distance = current_distance + 1  # Using unit edge costs
            
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_vertex
                heapq.heappush(pq, (distance, neighbor))
    
    # Reconstruct path
    path = []
    current = goal_idx
    while current in predecessors:
        path.append(current)
        current = predecessors[current]
    path.append(start_idx)
    path.reverse()
    
    return path if path[0] == start_idx else None

def execute_prm_path(sim, start_config, goal_config, n_samples=100, k_neighbors=5):
    """Plan path using PRM and prepare it for execution"""
    # Create PRM and find path
    roadmap, valid_samples, valid_indices, dummy_handles, line_handles = \
        create_prm_with_start_goal(sim, start_config, goal_config, n_samples, k_neighbors)
    
    # Find start and goal indices
    start_idx = valid_indices[-2]  # Second to last added node was start
    goal_idx = valid_indices[-1]   # Last added node was goal
    
    # Find path in roadmap
    path_indices = find_path_in_roadmap(roadmap, valid_indices, start_idx, goal_idx)
    
    if path_indices is None:
        print("No path found!")
        return None
    
    # Convert path indices to configurations
    path_configs = [valid_samples[valid_indices.index(idx)] for idx in path_indices]
    return path_configs, dummy_handles, line_handles

In [81]:
def main_loop(sim, joints, points, n_steps=400):
    """Main execution loop with PRM path following"""
    sim.setStepping(True)
    sim.startSimulation()
    
    # Initial setup
    js = read_joints(joints)
    ps = read_points(points)
    target_thetas = solve(js, ps)
    start_config = [0, 0, 0, 0, 0]
    
    # Plan path using PRM first
    path_configs, dummy_handles, line_handles = execute_prm_path(
        sim, start_config, target_thetas, n_samples=300, k_neighbors=5
    )
    
    if path_configs is None:
        print("Failed to find a path. Stopping execution.")
        sim.stopSimulation()
        return
    
    # Set up for execution
    place_thetas = [np.pi, -np.pi / 6, -np.pi / 2.7, -np.pi / 3, 0]
    base_thetas = [0, 0, 0, 0, 0]
    stage = 0
    gripper_positions = collections.deque(maxlen=100)
    current_path_index = 0
    
    # Main execution loop
    for i in range(n_steps):
        if stage == 0:  # Follow PRM path to target
            if current_path_index < len(path_configs):
                current_config = path_configs[current_path_index]
                diff_sum = trace_arm(joints, current_config)
                
                if abs(diff_sum) < 0.005:
                    current_path_index += 1
                    if current_path_index >= len(path_configs):
                        stage = 1
            
        elif stage == 1:  # grip target
            position = control_gripper(joints[-1], True)
            gripper_positions.append(position)
            if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
                stage = 2
                
        elif stage == 2:  # pick the target
            target_thetas[0] = np.pi
            target_thetas[1] = -np.pi / 12
            target_thetas[2] = -np.pi / 6
            diff_sum = trace_arm(joints, target_thetas)
            if abs(diff_sum) < 0.005:
                stage = 3
                
        elif stage == 3:  # move to place
            diff_sum = trace_arm(joints, place_thetas)
            if abs(diff_sum) < 0.005:
                stage = 4
                
        elif stage == 4:  # place the target
            position = control_gripper(joints[-1], False)
            gripper_positions.append(position)
            if len(gripper_positions) > 5 and np.abs(gripper_positions[-1] - gripper_positions[-5]) < 0.001:
                stage = 5
                
        elif stage == 5:  # move to base
            diff_sum = trace_arm(joints, base_thetas)
            if abs(diff_sum) < 0.005:
                stage = 6
                
        sim.step()
    
    # Clean up visualization
    for handle in dummy_handles:
        sim.removeObject(handle)
    for handle in line_handles:
        sim.removeDrawingObject(handle)
        
    sim.stopSimulation()



In [82]:
# Set up simulation parameters
n_steps = 400

# Run the main execution loop
main_loop(sim, joints, points, n_steps)

C:\Users\joy4m\AppData\Local\Temp\ipykernel_40664\3481707928.py:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  target_thetas = fsolve(


Checking samples for collision...
Start configuration added
Goal configuration added
Found 270 collision-free samples
Finding nearest neighbors...
Building and visualizing roadmap...
Roadmap construction completed


# Initialize

In [32]:
for i in range(len(joints)):
    sim.setJointPosition(joints[i], 0)
